Das Notebook macht eigentlich nichts spektakuläres, sondern füllt nur etwaige nicht vorhandene Werte. Dafür werden zwei Methoden angewendet:



*   Füllen basierend auf Wochentagen
*   Füllen basierend auf letzten Stunden

Für die Day Ahead Preise macht es z.B. Sinn, Wochentage zu betrachten, da diese sich zwischen den einzelnen Wochentagen (und vor allem zwischen Wochenende und Werktag) sehr stark voneinander unterscheiden. Für Wind Forecasts macht es Sinn, die vorherigen Stunden zu betrachten, da es keinen Sinn macht, vorherige Tage zu nehmen



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/01_Data_AI_SS24/Project_Turkish_Energy_Market/self_created_python_packages')

from database_functions import get_DataFrame_from_SQL_database, connect_to_database, write_dataframe_to_database

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
# connect to database
[conn_, cur_] = connect_to_database('original_data.db')

In [ ]:
startDate_ = "2021-01-01T00:00:00+03:00"
endDate_ = "2023-12-31T00:00:00+03:00"

startDate_formatted_ = startDate_[:10].replace("-","_")
endDate_formatted_ = endDate_[:10].replace("-","_")




In [ ]:
def find_null_indices(dataframe):
  """
  Funktion die Null Values findet und die indices zurückgibt
  """
  null_indices = {}
  for column in dataframe.columns:
    if dataframe[column].isnull().any():
      null_indices[column] = list(dataframe[dataframe[column].isnull()].index)
  return null_indices



In [ ]:
def update_null_values_value_before(dataframe, dictionary, previous_value = 1):
  """
  Funktion die einfach den vorherigen Indize Wert nimmt, um den für den Null Value zu nehmen
  """
  dataframe['date'] = pd.to_datetime(dataframe['date'], format="%Y-%m-%dT%H:%M:%S%z")
  for column, indices in dictionary.items():
      for index in indices:
          dataframe.at[index, column] = dataframe.at[index - previous_value, column]

  dataframe['date'] = dataframe['date'].dt.strftime("%Y-%m-%dT%H:%M:%S%z")
  return dataframe

In [ ]:
def update_null_values_daywise(dataframe, dictionary):
  """
  Funktion die den vorherigen Wochenwert nimmt, um die Null Values zu ersetzen
  """
  dataframe['date'] = pd.to_datetime(dataframe['date'], format="%Y-%m-%dT%H:%M:%S%z")
  for column, indices in dictionary.items():
      for index in indices:
          if dataframe.at[index, 'date'].weekday() in [0, 5, 6]:  # Monday, Saturday, Sunday
              previous_value = 7 * 24
          else:
              previous_value = 24

          mean_last_two_relevant_values = (dataframe.at[index - previous_value, column] + dataframe.at[index - 2 *previous_value, column])/ 2

          dataframe.at[index, column] = mean_last_two_relevant_values

  dataframe['date'] = dataframe['date'].dt.strftime("%Y-%m-%dT%H:%M:%S%z")
  return dataframe

In [ ]:
[conn_nonull_, cur_nonull_] = connect_to_database('data_no_null_values.db')

## Volume weighted average price

Spalten sind: Date, Hour, Weighted Avergae Price (MWh)


In [ ]:
weighted_avearge_prices_ = get_DataFrame_from_SQL_database(cur_, 'WEIGHTED_AVERAGE_PRICES', startDate_formatted_, endDate_formatted_)

null_indices_dict_ = find_null_indices(weighted_avearge_prices_)

updated_df_ = update_null_values_daywise(weighted_avearge_prices_, null_indices_dict_)

write_dataframe_to_database(updated_df_, table_name = f'WEIGHTED_AVERAGE_PRICES_{startDate_formatted_}_{endDate_formatted_}_NO_NULLVALUES', connection = conn_nonull_, cursor = cur_nonull_)

del null_indices_dict_, weighted_avearge_prices_, updated_df_

WEIGHTED_AVERAGE_PRICES_2021_01_01_2023_12_31_NO_NULLVALUES Table now in database, database not closed, please close later


## Day Ahead



**Day Ahead Preise (Market Clearing Prices)**

Spalten sind: Date, Hour, MCP (TL/MWh), MCP (USD/MWh), MCP (EUR/MWh)

In [ ]:
day_ahead_prices_ = get_DataFrame_from_SQL_database(cur_, 'DAY_AHEAD_PRICES', startDate_formatted_, endDate_formatted_)

null_indices_dict_ = find_null_indices(day_ahead_prices_)

updated_df_ = update_null_values_daywise(day_ahead_prices_, null_indices_dict_)

write_dataframe_to_database(updated_df_, table_name = f'DAY_AHEAD_PRICES_{startDate_formatted_}_{endDate_formatted_}_NO_NULLVALUES', connection = conn_nonull_, cursor = cur_nonull_)

del null_indices_dict_, day_ahead_prices_, updated_df_

DAY_AHEAD_PRICES_2021_01_01_2023_12_31_NO_NULLVALUES Table now in database, database not closed, please close later


**Day Ahead Trade Volumen**

Spalten sind: Date, Hour, Trade Volume (TL)

In [ ]:
day_ahead_volume_ = get_DataFrame_from_SQL_database(cur_, 'DAY_AHEAD_VOLUME', startDate_formatted_, endDate_formatted_)

null_indices_dict_ = find_null_indices(day_ahead_volume_)

updated_df_ = update_null_values_daywise(day_ahead_volume_, null_indices_dict_)

write_dataframe_to_database(updated_df_, table_name = f'DAY_AHEAD_VOLUME_{startDate_formatted_}_{endDate_formatted_}_NO_NULLVALUES', connection = conn_nonull_, cursor = cur_nonull_)

del null_indices_dict_, day_ahead_volume_, updated_df_

DAY_AHEAD_VOLUME_2021_01_01_2023_12_31_NO_NULLVALUES Table now in database, database not closed, please close later


**Geplante Produktion**

Spalten sind: Date, Hour,
Total (MWh),
Natural Gas,
Wind,
Lignite,
Black Coal,
Import Coal,
Fuel Oil,
Geothermal,
Dam,
Naphtha,
Biomass,
River,
Other

In [ ]:
planned_production_ = get_DataFrame_from_SQL_database(cur_, 'PLANNED_PRODUCTION', startDate_formatted_, endDate_formatted_)

null_indices_dict_ = find_null_indices(planned_production_)

updated_df_ = update_null_values_daywise(planned_production_, null_indices_dict_)

write_dataframe_to_database(updated_df_, table_name = f'PLANNED_PRODUCTION_{startDate_formatted_}_{endDate_formatted_}_NO_NULLVALUES', connection = conn_nonull_, cursor = cur_nonull_)

del null_indices_dict_, planned_production_, updated_df_

PLANNED_PRODUCTION_2021_01_01_2023_12_31_NO_NULLVALUES Table now in database, database not closed, please close later



**Tatsächliche Einspeisung**

Spalten sind: Date, Hour,
Total (MWh),
Natural Gas,
Dam,
Lignite,
River,
Import Coal,
Solar,
Wind,
Fuel Oil,
Geothermal,
Asphaltite Coal,
Black Coal,
Biomass,
Naphtha,
LNG,
International Import,
International Export,
Other

In [ ]:
injection_quanitity_ = get_DataFrame_from_SQL_database(cur_, 'INJECTION_QUANTITY', startDate_formatted_, endDate_formatted_)

null_indices_dict_ = find_null_indices(injection_quanitity_)

updated_df_ = update_null_values_daywise(injection_quanitity_, null_indices_dict_)

write_dataframe_to_database(updated_df_, table_name = f'INJECTION_QUANTITY_{startDate_formatted_}_{endDate_formatted_}_NO_NULLVALUES', connection = conn_nonull_, cursor = cur_nonull_)

del null_indices_dict_, injection_quanitity_, updated_df_

INJECTION_QUANTITY_2021_01_01_2023_12_31_NO_NULLVALUES Table now in database, database not closed, please close later


**Forecast Load**

Spalten sind: Date, Hour, Load Forecast Plan (MWh)



In [ ]:
forecast_load_ = get_DataFrame_from_SQL_database(cur_, 'FORECAST_LOAD', startDate_formatted_, endDate_formatted_)

null_indices_dict_ = find_null_indices(forecast_load_)

updated_df_ = update_null_values_daywise(forecast_load_, null_indices_dict_)

write_dataframe_to_database(updated_df_, table_name = f'FORECAST_LOAD_{startDate_formatted_}_{endDate_formatted_}_NO_NULLVALUES', connection = conn_nonull_, cursor = cur_nonull_)

del null_indices_dict_, forecast_load_, updated_df_

FORECAST_LOAD_2021_01_01_2023_12_31_NO_NULLVALUES Table now in database, database not closed, please close later


**Realtime Consumption**

Spalten sind: Date, Hour, Consumption Quantitiy (MWh)

In [ ]:
real_time_consumption_ = get_DataFrame_from_SQL_database(cur_, 'REAL_TIME_CONSUMPTION', startDate_formatted_, endDate_formatted_)

null_indices_dict_ = find_null_indices(real_time_consumption_)

updated_df_ = update_null_values_daywise(real_time_consumption_, null_indices_dict_)

write_dataframe_to_database(updated_df_, table_name = f'REAL_TIME_CONSUMPTION_{startDate_formatted_}_{endDate_formatted_}_NO_NULLVALUES', connection = conn_nonull_, cursor = cur_nonull_)

del null_indices_dict_, real_time_consumption_, updated_df_

REAL_TIME_CONSUMPTION_2021_01_01_2023_12_31_NO_NULLVALUES Table now in database, database not closed, please close later


## Failure Information
Only Check for missing values


In [ ]:
planned_failure_ = get_DataFrame_from_SQL_database(cur_, 'PLANNED_FAILURE', startDate_formatted_, endDate_formatted_)

null_indices_dict_ = find_null_indices(planned_failure_)

print(f'Planned Information has {len(null_indices_dict_)} missing values')

write_dataframe_to_database(planned_failure_, table_name = f'PLANNED_FAILURE_{startDate_formatted_}_{endDate_formatted_}_NO_NULLVALUES', connection = conn_nonull_, cursor = cur_nonull_)

del planned_failure_, null_indices_dict_

Planned Information has 0 missing values
PLANNED_FAILURE_2021_01_01_2023_12_31_NO_NULLVALUES Table now in database, database not closed, please close later


In [ ]:
unplanned_failure_ = get_DataFrame_from_SQL_database(cur_, 'UNPLANNED_FAILURE', startDate_formatted_, endDate_formatted_)

null_indices_dict_ = find_null_indices(unplanned_failure_)

print(f'Unplanned Information has {len(null_indices_dict_)} missing values')

write_dataframe_to_database(unplanned_failure_, table_name = f'UNPLANNED_FAILURE_{startDate_formatted_}_{endDate_formatted_}_NO_NULLVALUES', connection = conn_nonull_, cursor = cur_nonull_)

del unplanned_failure_, null_indices_dict_

Unplanned Information has 0 missing values
UNPLANNED_FAILURE_2021_01_01_2023_12_31_NO_NULLVALUES Table now in database, database not closed, please close later


## WPP Forecast and Generation

Spalten sind: Hour,
Forecast (MW),
Generation (MW)

In [ ]:
wpp_generation_and_forecast_ = get_DataFrame_from_SQL_database(cur_, 'WPP_GENERATION_AND_FORECAST', startDate_formatted_, endDate_formatted_)

null_indices_dict_ = find_null_indices(wpp_generation_and_forecast_)

updated_df_ = update_null_values_value_before(wpp_generation_and_forecast_, null_indices_dict_)

write_dataframe_to_database(updated_df_, table_name = f'WPP_GENERATION_AND_FORECAST_{startDate_formatted_}_{endDate_formatted_}_NO_NULLVALUES', connection = conn_nonull_, cursor = cur_nonull_)

del null_indices_dict_, wpp_generation_and_forecast_, updated_df_

WPP_GENERATION_AND_FORECAST_2021_01_01_2023_12_31_NO_NULLVALUES Table now in database, database not closed, please close later


## Balance Market

**System's Direction**

Spalten sind: Date, Hour, System Direction (Energy Surplus, Balanced, Energy Deficit)

In [ ]:
system_direction_ = get_DataFrame_from_SQL_database(cur_, 'SYSTEM_DIRECTION', startDate_formatted_, endDate_formatted_)

null_indices_dict_ = find_null_indices(system_direction_)

updated_df_ = update_null_values_value_before(system_direction_, null_indices_dict_)

write_dataframe_to_database(updated_df_, table_name = f'SYSTEM_DIRECTION_{startDate_formatted_}_{endDate_formatted_}_NO_NULLVALUES', connection = conn_nonull_, cursor = cur_nonull_)

del null_indices_dict_, system_direction_, updated_df_

SYSTEM_DIRECTION_2021_01_01_2023_12_31_NO_NULLVALUES Table now in database, database not closed, please close later


Check up- and down regulation

In [ ]:
up_regulation_= get_DataFrame_from_SQL_database(cur_, 'UP_REGULATION', startDate_formatted_, endDate_formatted_)

null_indices_dict_ = find_null_indices(up_regulation_)

print(f'Up Regulation has {len(null_indices_dict_)} missing values')

write_dataframe_to_database(up_regulation_, table_name = f'UP_REGULATION_{startDate_formatted_}_{endDate_formatted_}_NO_NULLVALUES', connection = conn_nonull_, cursor = cur_nonull_)

del up_regulation_, null_indices_dict_

Up Regulation has 0 missing values
UP_REGULATION_2021_01_01_2023_12_31_NO_NULLVALUES Table now in database, database not closed, please close later


In [ ]:
down_regulation_ = get_DataFrame_from_SQL_database(cur_, 'DOWN_REGULATION', startDate_formatted_, endDate_formatted_)

cur_nonull_.execute(f"DROP TABLE IF EXISTS DOWN_REGULATION_{startDate_formatted_}_{endDate_formatted_}")

null_indices_dict_ = find_null_indices(down_regulation_)

print(f'Down Regulation has {len(null_indices_dict_)} missing values')

write_dataframe_to_database(down_regulation_, table_name = f'DOWN_REGULATION_{startDate_formatted_}_{endDate_formatted_}_NO_NULLVALUES', connection = conn_nonull_, cursor = cur_nonull_)

del down_regulation_, null_indices_dict_

Down Regulation has 0 missing values
DOWN_REGULATION_2021_01_01_2023_12_31_NO_NULLVALUES Table now in database, database not closed, please close later


Power Plant failure

In [ ]:
power_plant_fail_ = get_DataFrame_from_SQL_database(cur_, 'FAILURE_MESSAGE', startDate_formatted_, endDate_formatted_)

cur_nonull_.execute(f"DROP TABLE IF EXISTS FAILURE_MESSAGE_{startDate_formatted_}_{endDate_formatted_}_NO_NULLVALUES")

null_indices_dict_ = find_null_indices(power_plant_fail_)

print(f'FAILURE_MESSAGE has {len(null_indices_dict_)} missing values')

write_dataframe_to_database(power_plant_fail_, table_name = f'FAILURE_MESSAGE_{startDate_formatted_}_{endDate_formatted_}_NO_NULLVALUES', connection = conn_nonull_, cursor = cur_nonull_)

#del power_plant_fail_, null_indices_dict_

FAILURE_MESSAGE has 0 missing values
FAILURE_MESSAGE_2021_01_01_2023_12_31_NO_NULLVALUES Table now in database, database not closed, please close later


In [ ]:
power_plant_fail_

,orgName,powerPlantName,uevcbName,caseStartDate,caseEndDate,operatorPower,capacityAtCaseTime,reason,id,uevcbId
0,ELEKTRİK ÜRETİM A.Ş.,ILISU BARAJI ve HES,ILISU BARAJI ve HES,2023-11-28T00:00:00+03:00,2023-11-28T21:59:00+03:00,1208.60,0,BOĞULMA VAKASI.,356973,3211210
1,ADAPAZARI ELEKTRİK ÜRETİM LİMİTED ŞİRKETİ,ADAPAZARI DGKÇS,ADAPAZARI DGKÇS,2022-09-30T00:00:00+03:00,2022-09-30T23:59:00+03:00,818.00,0,Buhar Türbini - IP Türbin Arızası,313679,3205381
2,SİLOPİ ELEKTRİK ÜRETİM A.Ş,SİLOPİ TES,SİLOPİ TES 154kV,2022-08-28T00:08:00+03:00,2022-08-28T23:59:00+03:00,135.00,0,1.Ünite türbin arızasından dolayı servis harici.,309896,3964
3,ELEKTRİK ÜRETİM A.Ş.,TEKİRDAĞ DGKÇS_B(Doğalgaz/Motorin),TEKİRDAĞ DGKÇS_B(Doğalgaz/Motorin),2021-04-15T10:40:00+03:00,2021-04-15T10:59:00+03:00,478.00,350,START ÇALIŞMASI,246078,3204399
4,YEŞİLYURT ENERJİ ELEKTRİK ÜRETİM A.Ş.,SAMSUN OSB DGKÇS,Samsun OSB DGKÇS,2021-10-08T00:00:00+03:00,2021-10-08T23:59:00+03:00,234.45,216,7 Nolu makina yağlama pompası arızası nedeniyl...,267586,335813
...,...,...,...,...,...,...,...,...,...,...
93336,AKSA GÖYNÜK ENERJİ ÜRETİM ANONİM ŞİRKETİ,BOLU-GÖYNÜK ENERJİ SANTRALİ,BOLU-GÖYNÜK ENERJİ SANTRALİ_1,2021-01-05T03:00:00+03:00,2021-01-05T04:59:00+03:00,135.00,83,kül sevk sistemi arızası,232458,3192481
93337,KÜÇÜK ENERJİ ÜRETİM VE TİCARET LİMİTED ŞİRKETİ,KÖPRÜBAŞI HES(KÜÇÜK ENR.),KÖPRÜBAŞI HES(KÜÇÜK ENR.),2022-10-05T00:05:00+03:00,2022-10-05T23:59:00+03:00,7.95,0,İletim kanalında göçük,314198,3194111
93338,GARET ENERJİ ÜRETİM VE TİCARET A.Ş.,SARES RES,SARES RES,2022-03-29T02:41:00+03:00,2022-03-29T12:59:00+03:00,27.50,25,1 TÜRBİN ARIZA,291735,26808
93339,ABK ENERJİ ELEKTRİK ÜRETİM A.Ş.,Söke-Çatalbük Rüzgar Elektrik Santrali,SÖKE ÇATALBÜK,2021-10-11T00:10:00+03:00,2021-10-11T23:59:00+03:00,30.00,26,Türbin ARızası,268007,81786


In [ ]:
conn_nonull_.commit()
conn_nonull_.close()

In [ ]:
get_DataFrame_from_SQL_database(cur_, 'FAILURE_MESSAGE', startDate_formatted_, endDate_formatted_)


,orgName,powerPlantName,uevcbName,caseStartDate,caseEndDate,operatorPower,capacityAtCaseTime,reason,id,uevcbId
0,ELEKTRİK ÜRETİM A.Ş.,ILISU BARAJI ve HES,ILISU BARAJI ve HES,2023-11-28T00:00:00+03:00,2023-11-28T21:59:00+03:00,1208.60,0,BOĞULMA VAKASI.,356973,3211210
1,ADAPAZARI ELEKTRİK ÜRETİM LİMİTED ŞİRKETİ,ADAPAZARI DGKÇS,ADAPAZARI DGKÇS,2022-09-30T00:00:00+03:00,2022-09-30T23:59:00+03:00,818.00,0,Buhar Türbini - IP Türbin Arızası,313679,3205381
2,SİLOPİ ELEKTRİK ÜRETİM A.Ş,SİLOPİ TES,SİLOPİ TES 154kV,2022-08-28T00:08:00+03:00,2022-08-28T23:59:00+03:00,135.00,0,1.Ünite türbin arızasından dolayı servis harici.,309896,3964
3,ELEKTRİK ÜRETİM A.Ş.,TEKİRDAĞ DGKÇS_B(Doğalgaz/Motorin),TEKİRDAĞ DGKÇS_B(Doğalgaz/Motorin),2021-04-15T10:40:00+03:00,2021-04-15T10:59:00+03:00,478.00,350,START ÇALIŞMASI,246078,3204399
4,YEŞİLYURT ENERJİ ELEKTRİK ÜRETİM A.Ş.,SAMSUN OSB DGKÇS,Samsun OSB DGKÇS,2021-10-08T00:00:00+03:00,2021-10-08T23:59:00+03:00,234.45,216,7 Nolu makina yağlama pompası arızası nedeniyl...,267586,335813
...,...,...,...,...,...,...,...,...,...,...
93336,AKSA GÖYNÜK ENERJİ ÜRETİM ANONİM ŞİRKETİ,BOLU-GÖYNÜK ENERJİ SANTRALİ,BOLU-GÖYNÜK ENERJİ SANTRALİ_1,2021-01-05T03:00:00+03:00,2021-01-05T04:59:00+03:00,135.00,83,kül sevk sistemi arızası,232458,3192481
93337,KÜÇÜK ENERJİ ÜRETİM VE TİCARET LİMİTED ŞİRKETİ,KÖPRÜBAŞI HES(KÜÇÜK ENR.),KÖPRÜBAŞI HES(KÜÇÜK ENR.),2022-10-05T00:05:00+03:00,2022-10-05T23:59:00+03:00,7.95,0,İletim kanalında göçük,314198,3194111
93338,GARET ENERJİ ÜRETİM VE TİCARET A.Ş.,SARES RES,SARES RES,2022-03-29T02:41:00+03:00,2022-03-29T12:59:00+03:00,27.50,25,1 TÜRBİN ARIZA,291735,26808
93339,ABK ENERJİ ELEKTRİK ÜRETİM A.Ş.,Söke-Çatalbük Rüzgar Elektrik Santrali,SÖKE ÇATALBÜK,2021-10-11T00:10:00+03:00,2021-10-11T23:59:00+03:00,30.00,26,Türbin ARızası,268007,81786
